<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>切入点 2：</b>工具驱动的 LLM 系统</h2>
<br>

**欢迎来到我们的第二个扩展内容！**

在之前的 Notebook 中，我们强调了模型的严格限制，并实验了压榨系统生成能力的方法。我们能够接近有趣的输出需求，技术上可以让 LLM 以一致和结构化的方式进行交互，甚至能够输出较长的文档。这部分，我们将研究工具（tooling），以及能让 LLM 在环境中进行操作的特性。

### **学习目标：**
**在这个 Notebook 中，我们将：**

- 介绍一些 LLM 调度技术，这些技术源自我们新发现的生成结构化输出的能力。
- 研究工具这个概念，看看为何定义和区分对您选择的抽象具有重要意义。

In [ ]:
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

<hr><br>

## **第一部分：** 探索可控的环境

现在我们对如何用 LangGraph 实现一个智能体事件循环有了一些了解，是时候把这项技能用上了！第 3 次练习中已经初次体会到了智能体，这个过程在某种程度上很容易和第一部分相关联，但也为智能体的实现过程提供了一个很简单的框架。

- **将智能体放入对话循环中。**
- **强制它们输出生成变量的结构。**
- **根据生成的变量，调整控制流程。**

我们根据控制流程的具体行为起了不同的名称：
- 当这个控制流程用于**选择工具或路径**时，被称为***路由（Routing）***。
- 当它用于**选择和参数化**工具（假设会被调用）时，被称为***工具化（Tooling）***。
- 当它用于**检索信息**时，这被称为***检索（Retrieval）***。

您可能会细心地注意到这些术语之间没有具体的区别；只有语义上的不同。尽管如此，定义这些特性并考虑您想如何区分它们，无论是在思维、编码中，还是在沟通系统的努力和特性时，都是非常有用的。

<img src="images/llm-orchestration.png" style="width:1000px;"/>

<br>

#### **与 LLM 持续面临的挑战**

尽管可以轻松地想象一个具有人类技能的 LLM 如何利用我们的要求与各种资源进行交互，但必须记住与 LLM 相关的各种怪癖。
- 它们很容易被引导偏，表现出其训练方法、数据、输入风格和整体能力。
- 它们的实现各不相同，包含各种假设、默认支持机制以及不同（有时怀疑的）机制实现。

这造成了一种有趣的动态：如果您希望它们共同合作解决非平凡的任务，那么您需要对我们的 LLM 及其真正能力做出一些假设。
- 它们能调用工具吗？能够路由到工具吗？能提出好的问题吗？
- 它们能理解整个环境吗？甚至是对话或最后一条消息呢？
- 它们应该立即响应吗？还是计划和执行？或依赖其他系统？

根据您的观察，您的模型池和预算将强烈决定一个真正的多智能体工作流是否真的有用。我们将把讨论限制在 Llama-8B 模型类别上（您可能会认为这并不好，对吧？），然后看看我们能从中获得什么。

<hr><br>

## **第二部分：** 识别一些工具

我们已经学习了结构化输出，正朝着实现某种路由的方向走。然而，您需要根据可以访问的模型及预期的使用场景选择合适的 API。这里有一些在实际场景中可能会遇到的常见情况：

- **闭源 LLM：** 大多数无法获取源代码的 LLM 提供商尝试支持智能工作流，虽然不一定会宣传他们真正的模型设置。这就是为什么许多 LLM 入口不再支持原始的 `/completions` 入口，而是选择标准化的 `/chat/completions` 入口。
    - 这意味着为了支持工具，您必须遵循他们的工具/结构化输出 API，并希望它能正常工作。（**OpenAI [Function](https://platform.openai.com/docs/guides/function-calling)/[Assistants](https://platform.openai.com/docs/assistants/tools) API，Claude [Tool Use API](https://docs.anthropic.com/en/docs/build-with-claude/tool-use)**）
    - 实际上，这通常非常好，并且可能会有一些后台的优化，包括自动提示词注入、服务器端拒绝和缓存。
- **开源 LLM：** 许多开源项目旨在标准化和统一社区的 API 抽象，以帮助人们交流并找到最适合他们目的的模型。因此，社区还创建了类似的支持项目，帮助开发与私有服务器隐藏选项竞争的工具。
    - **从表面来看，** 这表现为对最流行的 API 的支持，仅在必要时偏离。因此，几乎所有解决方案都支持 LLM、VLM 和嵌入模型的 OpenAI API 规格，而只有一些标准化已针对扩散、重排序和文档摄取 API 进行了处理。
    - **从更深层次来看，** 这种接口的支持是一种尽力而为的尝试，偶尔可能会与模型训练相对立，或者将模型应用到超出其推荐的范围。

因此，我们将查看几种可能的配置，既包括支持它的客户端抽象，也假设包括满足其责任的服务器后台。

<br>

### **第二部分 1：客户端工具**

像 LangChain 这样的框架为感兴趣的人提供了工具接口。如果您想将一个功能集成到 LLM 中，能够不必编写一大堆样板代码来使其正常工作。下面，我们将看到使用 `@tool` 装饰器定义“工具”的 LangChain 方式。

In [ ]:
from langchain.tools import tool
from typing import List, Literal

@tool
def calculate(
    thought_process: List[str],
    # tool: Literal["add"],
    # tool: Literal["add", "mult", "pow"],
    tool: Literal["add", "subtract", "multiply", "divide", "power"],
    a: float, 
    b: float
) -> int:
    """Adds a and b. Requires both arguments."""
    if tool == "add": return a + b
    if tool == "subtract": return a - b
    if tool == "multiply": return a * b
    if tool == "divide": return a / b
    if tool == "power": return a ** b

print(calculate.name)
print(calculate.description)
print(calculate.args)
calculate.input_schema.model_json_schema()

<br>

如您所见，这只是一层轻量的封装，围绕着模式抽象，允许它们构建工具。使用与之前相同的策略，我们可以继续以可预测的方式调用这个工具：

In [ ]:
from course_utils import SCHEMA_HINT

sys_msg = (
    "You are a world-class calculator. Please answer the user's question, and use your tools."
    # "Think through your decision in thought-process until you know your first step using order-of-operations. "
    # "Predict the first tool as your last output. Be specific, and then call the tool."
)
# sys_block = []
# sys_block = [("system", sys_msg)]
schema_hint = SCHEMA_HINT.format(schema_hint=calculate.input_schema.model_json_schema())
sys_block = [("system", f"{sys_msg}\n\n{schema_hint}")]

# question = "What's 56464 + 4789789097?"
# question = "What's 56464 - 4789789097?"
# question = "What's 56464 / 4789789097?"
question = "What's 56464 / 4789789097 + 6750 * 478978090?"

calc_llm = llm.with_structured_output(calculate.input_schema)
a = calc_llm.invoke(sys_block + [("user", question)])
print(a)

In [ ]:
calc_tool = llm.with_structured_output(calculate.input_schema) | dict | calculate
calc_tool.invoke(sys_block + [("user", question)])

In [ ]:
llm._client.last_inputs

您可以看到，这个版本的 API 只是对结构化输出格式的一种轻微加工，额外添加了一些便利功能。虽然没有特别出彩，但了解有这样一个简化的 API 还是不错的。

<br>

### **第二部分 2：服务器端工具选择**

相比之下，服务器端工具选择不仅仅是代码的简化。许多支持结构化输出接口的入口也试图支持一个明确的工具选项接口，这使得 LLM 能选择一些工具进行调用。这个实现的具体机制各不相同，所以您的入口可能支持以下几种配置：
- **强制工具调用**：通过强制语法来强制选择一个类别，然后生成相应的模式。
    - **缺点：** 根据训练和强制，这可能会使 LLM 超出领域，因为这可能与模型训练相悖。
    - **好处：** 从生成的 token 和浪费的 token 的角度来看，这在技术上更高效。为这种类型的工具策划精细调整数据也相对简单。
- **非结构化输出 -> 工具调用**：允许 LLM 生成一些输出（可能是推理，也可能是随意对话）。这些材料可以被丢弃，作为响应消息体输出，或者以其他方式集成到结构化输出中。之后，结构化输出（经过指导解码或其他方式强制）被聚合并返回给用户。
    - **缺点：** 生成了更多的 token，具体取决于实现（无论是服务器端还是客户端），附加的 token 可能默认被丢弃。
    - **好处：** 更可能在领域内，并且可能允许超越预期模式的更深层推理。此外，可能还允许对话式工具调用（谈论调用，接着发出调用，然后自然地结束对话）。

接下来，我们可以看到几个工具的定义，预先在生成的文档字符串上进行了隐式提示工程，通过额外的变量、合理的函数名称和附加的文档字符串。`search_knowledge` 的实现没有讨论，会在后续的 Notebook 中进行说明。

In [ ]:
from pydantic import Field
from langchain.tools import tool
from typing import Dict, List, Literal
import numpy as np

from contextlib import redirect_stdout

@tool
def search_internet(user_question: List[str], context: List[str], final_query: str):
    """Search the internet for answers. Powered by search engine, in Google search format."""
    from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
    return DuckDuckGoSearchAPIWrapper(backend="html").results(final_query, max_results=10, source="text")

@tool
def search_knowledge(user_question: List[str], context: List[str], final_query: str):
    """Search your knowledge for answers. Includes chat history, common responses, and directives"""
    return "No knowledge ingested. Respond in best-effort based on directive."

LockedImports = Literal["import numpy as np; import pandas as pd; import math; import print"]

@tool
def execute_python(user_question: List[str], context: List[str], imports: LockedImports, final_code: str):
    """Execute python code, the values printed through stdout (i.e. `print` will be returned to user)"""
    import contextlib, io
    import numpy as np; import pandas as pd; import math 
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        try: 
            exec(final_code, {"numpy": np, "np": np, "pandas": pd, "pd": pd, "math": math})
            return buf.getvalue()
        except Exception as e: 
            return str(e)

@tool
def random_choice(options: List[str], num_samples: int, probs: List[float]):
    """Returns a random option generated from the p distributions (list of floats)"""
    return np.random.choice(options, num_samples, probs)

schema = search_internet
schema = search_knowledge
schema = execute_python
print(schema.name)
print(schema.description)
print(schema.args)
schema.input_schema.model_json_schema()

<br>

幸运的是，我们的 Llama NIM 自带支持这个工具调用（您可以根据响应速度猜一下是基于哪种策略）。由于 LangChain 连接器通过 `bind_tools` 简化了工具绑定，我们可以像使用 `with_structured_output` 一样用它，然后只需给提示词添加一些模式提示就能让这一切顺利运行。
- **提醒：** 在 LangChain 中，`bind` 仅仅是将一个参数绑定到运行时/客户端。`bind_tools` 这类函数和 `with_structured_output` 是一种变体，它同样处理模式，使其符合服务器期望的格式。

In [ ]:
from course_utils import SCHEMA_HINT

toolbank = [search_internet, search_knowledge, calculate, execute_python, random_choice]
# toolbank = [search_internet, search_knowledge, calculate, execute_python]
# toolbank = [search_internet, search_knowledge, calculate]
# toolbank = [search_internet, search_knowledge]
tooldict = {tool.name: tool for tool in toolbank}
tool_arg_generator = llm.bind_tools(toolbank) | dict

query = (
    # "Can you please calculate the first 20 digits of pi?"
    # "Can you please calculate the first 20 digits of pi? Make sure to use the execute_python tool."
    # "Can you please pick a random color for me?"
    # "Can you please pick a random color for me with uniform probability?"
    "Can you please tell me about NVIDIA's new DIGITS systems?"
    # "Can you please tell me about NVIDIA's new DIGITS systems? What do you know from your knowledge?"
)

output = tool_arg_generator.invoke([
    ("system", (
        "You are an NVIDIA Chatbot. Please help the user with their concerns.\n"
        + SCHEMA_HINT.format(schema_hint="\n".join(str(tool.input_schema.model_json_schema()) for tool in toolbank))
    )), ("user", query),
])
# print(output)
print("Content:", output.get('content') or "Blank")
output.get('tool_calls', [])

In [ ]:

## Feel free to try some of these prompts out to see what works and what doesn't. 
## When you're ready to see if it would have worked in calling the tool, you can run the cell below:

for tc in output.get('tool_calls', []):
    print(tooldict[tc.get("name")].invoke(input=tc.get("args")))

<br>

接下来看看到工具的实际使用。从这个小例子中，您会注意到它并不完美，还需要像其他所有东西一样进行提示工程：
- 如果函数名称不够描述性，它可能默认使用一些更广泛的名词，比如“计算”，即使实际实现的功能不足。
- 对于轻量级模型，Python 运行工具实际上实现起来非常棘手，您可以看到我们通过添加使用库的提示来破解，使这个简单调用至少稳定。
- 即使是不经意的措辞不匹配也会导致工具使用不当，如果您的系统没有正确将指令转换为一些规范形式。

不过，看起来确实可以用 LLM 调用工具，甚至在需要的时候进行工具选择，这真的很酷！

请注意，从 `with_structured_output` 到 `bind_tools` 只是将工具选择的责任从客户端转移到了服务器。正如之前所说的，这不是一个表面的变化，实际上突出了某些优缺点。虽然 `bind_tools` 让调用者更容易使用，但也减少了控制，这可能对某些关键功能来说是必要的。在实现过程中，考虑这一点至关重要，并在从一个模型切换到另一个模型时选择正确的策略，因为服务器端的假设可能并不总是适用于特定的使用案例。




<hr><br>

## **第三部分：** 在循环中使用工具（ReAct）

现在我们定义了一些简单的工具来与环境进行交互，我们可以用这些工具来增强我们的 LLM，或许能期待一种多轮对话。实际上，甚至可能是一个多步骤对话，期间它们会使用多个工具，等到真的得到了答案再回来跟我们说。

> <img src="images/react-opts.png" style="width: 1000px" />
>
> <a href="https://react-lm.github.io/" target="_blank"><b>ReAct: Synergizing Reasoning and Acting in Language Models (2022)</b></a>
>

为了实现这一点，实际上有一些相当简单的方法可以处理这个问题，虽然这些方法都有一些失败模式，但似乎在模型不断改进的情况下，性能也能很好地扩展。我们将在这个 Notebook 中探讨其中的几个选项，但请放心，这些只是众多潜在选项中最流行的几种：

### **原始 ReAct：** 

ReAct 是“推理与行动”的缩写，这是一种经历了快速普及的技术，在 LLM 协同组织领域迅速发展，远远超出了最初的定义，随着框架的不断演变而变化。ReAct 最初被提出来作为维护**智能体记事本**的策略，在这个记事本中，LLM 会接收到一个指令，一些工具调用的示例，以及一些满足条件的示例。基于此，随着这些 `{questions,answers,fulfillment}` 的示例在窗口中堆积，语境会不断增长。这与仅仅使用 `{question,fulfillment}` 不同，因为答案首先会提供一些决策背后的推理。

例如，以下就是一个合理的默认提示指令：

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
print(prompt.template)

<br>

而使用这种默认提示词的结果会是直接来自 [遗留文档](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/) 的一个例子，应用于 8B 3.1 模型：

In [ ]:
import langchain.agents as lc_agents
from IPython.display import display
from langchain_nvidia import NVIDIA

base_llm = NVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

lc_agent_llm = base_llm

# Construct the ReAct agent
agent = lc_agents.create_react_agent(lc_agent_llm, toolbank, prompt)
agent_executor = lc_agents.AgentExecutor(agent=agent, tools=toolbank, verbose=True, handle_parsing_errors=True)

try:
    agent_executor.invoke(
        {
            "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
            # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
            "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
        },
        verbose=True
    )
except Exception as e:
    print("Exception:", e)
    print("\nInput:") or display(lc_agent_llm._client.last_inputs)
    print("\nOutput:") or display(getattr(lc_agent_llm._client.last_response, "json", lambda: None)(), None)

<br>

这个版本是个不错的开始，出现了许多有趣的例子，尤其是以没有指导的自回归作为主要技术时。好的例子会在上下文中加强工具调用，坏的例子则会导致投诉回传给 LLM，而 token 停止条件会确保 LLM 不会试图回答它积累起来的问题。

### **现代化的 ReAct：**

由于 ReAct的理念与智能体抽象紧密相连，它基本上演变成了一种更通用的想法，即“思考，交互，看看会发生什么，然后再来”。结合工具选择和结构化输出的涌现，这个术语也演变为涵盖任何智能系统，它：

- **有一个中心对话循环。**
- **可以调用可用的工具。**
- **可以直接响应用户。**

换句话说，**一个 ReAct 智能体现在只是任何拥有运行对话缓冲区的智能体，并且可以调用包括用户在内的一组工具。**您可以自由地讨论这是否是一个合理的演变，但确实听起来挺有意思的，并且这个术语也就保留下来了。

我们可以尝试在 `langgraph` 库中以其默认导入形式来试试这种新风格的 ReAct 循环，看看它的调用结果会有所不同：

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.history import RunnableWithMessageHistory

checkpointer = MemorySaver()
langgraph_agent_executor = create_react_agent(llm, toolbank, checkpointer=checkpointer)

query = (
    # "Can you please calculate the first 20 digits of pi?"
    "Can you please calculate the first 20 digits of pi? Make sure to use the execute_python tool."
    # "Can you please pick a random color for me?"
    # "Can you please pick a random color for me with uniform probability?"
    # "Can you please tell me about NVIDIA's new DIGITS systems?"
    # "Can you please tell me about NVIDIA's new DIGITS systems? What do you know from your knowledge?"
)

config = {"configurable": {"session_id": "test-session", "thread_id": "test-thread"}}

messages = langgraph_agent_executor.invoke({"messages": [("human", query)]}, config=config)
{"input": query, "output": messages["messages"][-1].content}

我们可以查看历史记录，似乎... 它的表现如预期一样：

In [ ]:
langgraph_agent_executor.get_state(config).values

<br>

这是因为这一版本强烈假设可以通过工具使用 API 来调用工具，并且只是将现已被称为“ReAct 循环”的内容作为其主要控制流程来执行：

In [ ]:
langgraph_agent_executor

<br>

不过，您可能还会注意到，这个循环在多轮对话的上下文中并不能正常工作：

In [ ]:
messages = langgraph_agent_executor.invoke({"messages": [("human", "Can you now see what happens when you use another tool?")]}, config=config)
messages["messages"][-1].content

In [ ]:
langgraph_agent_executor.get_state(config).values

<br>

可以自由地推测原因，但这绝对与 `ToolMessage` 如何被服务器处理有关。出于这个原因以及其他许多原因，这种范式的默认实现需要为大多数用例重新构建，以适应特定 LLM 资源的期望、API 和优势。

<hr><br>

## **第四部分:** 服务器端工具执行

到目前为止，我们已经展示了客户侧工具选择的例子，并刚引入了服务器侧工具选择。在这两种情况下，入口仅负责问题的语义部分，而实际的实现仍然是客户侧的负担。但情况并不总是如此，也有一些有效的服务器端工具使用案例，甚至自动化工具支持。

> <img src="images/tooling-patterns.png" style="width:1000px;"/>

#### **包装应用/智能体：**

显然，您可以围绕一个 LLM 创建一个包装应用，带着自己的工具假设和实现策略。很多这样的系统的接口方式与 LLM 服务器大致相同（即您可以流式传输或调用 API），但会调用许多工具，管理自己的状态等。这并不让人惊讶，只是微服务设计模式的简单应用。

**例子：** 
- 一个**检索微服务**可能支持补全流式 API，并可能在内部组件中有结构化输出，即使它的主要任务是与某个知识库接口。
- 一个**聊天机器人智能体**可以简化为一个高级 API，该 API 自动维护状态并从其一系列预定义工具中选择，同时保持系统提示。如果它只在服务器上运行，其运行时可以移动到任意计算源，我们也可以定义任意资源所有权/扩展规则。

#### **测试时计算/推理时扩展：** 

有时工具、路由或分支策略与训练紧密相连，或者通过在后台实施来提升模型的表现。这种情况下，您可能会看到一些短语，如*“测试时计算（Test-Time Compute）”*和*“推理时扩展（Inference-Time Scaling）”*被提及，但定义不明确（可能被描述为“思考”）。这些都是含糊的术语，且其意义在不断演变，技术上是同义的：
- **测试时**和**推理时**意味着这些操作发生在模型已经训练完成之后，通常是在其实际运行的环境中。
- **计算**和**扩展**(或 *计算扩展*) 表示在模型的决策和输出生成中增加了额外的计算干预，强调了相当大的处理努力。

敏感的小伙伴可能会注意到，这听起来像是一个智能工作流或 LLM 工作流... 但现在它被认可为 LLM 推理的特性？从技术上讲，通常暗示着模型经过训练以支持这个额外的流程，或者通过许多合成数据，或整合了一个额外的专门为此过程设计的模型，但这并不是一个必需的条件。

**例子：**
- **“扩展：**一个输出思维链的推理系统，其输出可以自动扩展，以便每个过程要么**并行执行（*分支*）**，要么**顺序执行（*迭代*）**，或**合并在一起（*合并*）**。这可能会增加所需的推理时间，但可以被视为模型的固有特性，因为它专门为这种格式训练。
- **“计算：**这个类别相当广泛，可以包括任何进入推理服务器的 LLM 协同调度。举个例子，一个模型可能会**被某些分类器或奖励模型评估和判断**，以便在输出生成时使响应更为对齐。这可以用于使推理变得更重或更轻。
    - 对于较轻的选项，可以使用**猜测式解码（speculative decoding）**（与权重较轻的模型逐块自回归，直到遇到高不确定性的 token）和**动态保护措施**（用于分类的预训练或逐步训练嵌入模型）来加快推理。
    - 对于比较重的选项，使用**奖励模型进行批评和引导**通常会导致显著减速，但在关键场景中可能会非常有用。这有时用于推理，但在训练（即强化学习）中更为常见。

在这些情况中需要记住的是：

#### **工具注册：** 

对于那些利用复杂工作流（比如分支）同时限制对底层模型的访问的场景，通过网络接口支持并行化工具调用可能会相当具有挑战性。因此，一些更高级的工作流可能会将工具集限制为有限的预实现选项。

为了提供更多自定义，备选方案之一是让客户通过线程安全的入口（即可扩展/限制的异步方法）来托管自己的工具，并允许他们根据提供的模式注册可调用的工具。假设这个入口通过端口接口是可访问的，那么服务器可以异步调用托管的入口以满足请求。这种方法在 [**Anthropic Model Context Protocol**](https://www.anthropic.com/news/model-context-protocol) 中是显而易见的，技术上只是一个微服务风格的抽象，其中一个越来越边缘化的闭源服务器与一个更大功能生态系统进行交互。

<hr><br>

### **第五部分:** 反思本次练习

您可能会发现我们并没有把“工具”具体地抽象成某种形式。我们只是或多或少地将下面这两个陈述逻辑上合并得出了一个结论：

> **"一个 LLM 可以做出陈述和决策" + "一个 LLM 可以被迫输出可以被另一个系统解读的结构" = "您可以使用一个 LLM 系统的输出与另一个系统交互"**

接着我们还得出了另一个结论：您完全可以在不同的抽象层级上对这个过程进行模块化、卸载（offload）或者自动处理。这一点既基本又实用——正是通过这种方式，您可以：
- 完全绕开用户
- 有选择性地引入用户参与，
- 或者构建一个比传统“用户 + LLM 智能体”闭环更依赖 LLM 的极端系统。
而这甚至还只是触及了“系统如何自行定义控制流程”的表层：
- 您可以微调模型或使用嵌入模型来引导控制空间。
- 您可以有一个专家系统池，投票决定最佳路径，或者尝试以某种特定方向表达它们的理由和“信心”。
- 您可以有随机决策和算法条件逻辑——这本身就足够有趣，而且直觉上是显然可行的。

不过，这一切都是之前抽象的逻辑延伸，且高度依赖于 LLM 的特性和您愿意围绕这些特性进行工作的程度。